<H1>Problem Statement</H1>
Your client is a large MNC and they have 9 broad verticals across the organisation. One of the problem your client is facing is around identifying the right people for promotion (only for manager position and below) and prepare them in time. Currently the process, they are following is:

They first identify a set of employees based on recommendations/ past performance
Selected employees go through the separate training and evaluation program for each vertical. These programs are based on the required skill of each vertical
At the end of the program, based on various factors such as training performance, KPI completion (only employees with KPIs completed greater than 60% are considered) etc., employee gets promotion
For above mentioned process, the final promotions are only announced after the evaluation and this leads to delay in transition to their new roles. Hence, company needs your help in identifying the eligible candidates at a particular checkpoint so that they can expedite the entire promotion cycle.

They have provided multiple attributes around Employee's past and current performance along with demographics. Now, The task is to predict whether a potential promotee at checkpoint in the test set will be promoted or not after the evaluation process.

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!cp  /content/drive/Hackthon/WNS/* .
!ls -l

In [1]:
!ls -l


total 5556
-rw-r--r-- 1 root root    2495 Sep 15 07:51 adc.json
drwxr-xr-x 4 root root    4096 Sep 15 08:08 catboost_info
drwxr-xr-x 2 root root    4096 Sep 15 07:51 drive
drwxr-xr-x 2 root root    4096 Sep 13 17:28 sample_data
-rw-r--r-- 1 root root 1565147 Sep 15 08:57 test_2umaH9m.csv
-rw-r--r-- 1 root root 3759647 Sep 15 08:57 train_LZdllcl.csv
-rw-r--r-- 1 root root     229 Sep 15 08:57 Untitled
-rw-r--r-- 1 root root  211100 Sep 15 08:57 WNS_Traine_01.ipynb
-rw-r--r-- 1 root root   62371 Sep 15 08:57 WNS_Traine_02.ipynb
-rw-r--r-- 1 root root   57610 Sep 15 08:57 WNS_Traine_03.ipynb


In [0]:
!kill -9 2013 2028 2130 2145 2169 2154 


In [0]:
!pip install catboost
!pip install ipywidgets
!pip install lightgbm
!pip install xgboost     


<h1>3. Exploratory Data Analysis</h1>
<h3>3.1. Import All required Classed first</h3>


In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt


from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix,roc_auc_score,mean_squared_error,f1_score
from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.cross_validation import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from numpy import argmax
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool, cv
from lightgbm import LGBMClassifier

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [0]:
SEED = 0

In [0]:
Data = pd.read_csv("train_LZdllcl.csv")

In [5]:
print(Data.shape)
print(Data.columns)

(54808, 14)
Index(['employee_id', 'department', 'region', 'education', 'gender',
       'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'KPIs_met >80%', 'awards_won?',
       'avg_training_score', 'is_promoted'],
      dtype='object')


In [6]:
print(Data.describe())

        employee_id  no_of_trainings           age  previous_year_rating  \
count  54808.000000     54808.000000  54808.000000          50684.000000   
mean   39195.830627         1.253011     34.803915              3.329256   
std    22586.581449         0.609264      7.660169              1.259993   
min        1.000000         1.000000     20.000000              1.000000   
25%    19669.750000         1.000000     29.000000              3.000000   
50%    39225.500000         1.000000     33.000000              3.000000   
75%    58730.500000         1.000000     39.000000              4.000000   
max    78298.000000        10.000000     60.000000              5.000000   

       length_of_service  KPIs_met >80%   awards_won?  avg_training_score  \
count       54808.000000   54808.000000  54808.000000        54808.000000   
mean            5.865512       0.351974      0.023172           63.386750   
std             4.265094       0.477590      0.150450           13.371559   
min    

In [0]:
values={"no_of_trainings":0,"previous_year_rating":0,"gender":"O","education":"O"}
Data.fillna(value=values,inplace=True)

In [8]:
Data.isna().values.any()


False

In [9]:
print(len(Data))
print(len(Data.loc[Data["is_promoted"]==0]))
print(len(Data.loc[Data["is_promoted"]==1]))
print(len(Data))

54808
50140
4668
54808


In [10]:
50140/54808*100

91.48299518318494

In [11]:
Data.count()

employee_id             54808
department              54808
region                  54808
education               54808
gender                  54808
recruitment_channel     54808
no_of_trainings         54808
age                     54808
previous_year_rating    54808
length_of_service       54808
KPIs_met >80%           54808
awards_won?             54808
avg_training_score      54808
is_promoted             54808
dtype: int64

In [12]:
Data.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [0]:
data_xs = Data.drop(['employee_id',"is_promoted"],axis=1)
categorical_features_indices = np.array([0,1,2,3,4,6,8,10,11])
#data_xs = Data.drop(['employee_id',"is_promoted","education","gender","recruitment_channel","region"],axis=1)

In [0]:
data_xs_1 = Data.loc[:,["previous_year_rating","avg_training_score","length_of_service","no_of_trainings"]]
data_xs_2 = Data.loc[:,["previous_year_rating","avg_training_score"]]
data_xs_3 = Data.loc[:,["avg_training_score","no_of_trainings"]]
data_xs_4 = Data.loc[:,["avg_training_score","length_of_service"]]
data_xs_5 = Data.loc[:,["department","education","gender","recruitment_channel","region","age","KPIs_met >80%","awards_won?"]]
data_xs_6 = Data.loc[:,["department","KPIs_met >80%","avg_training_score","previous_year_rating"]]
data_xs_7 = Data.loc[:,["department","education","gender","recruitment_channel","region","age","awards_won?"]]
data_xs_8 = Data.loc[:,["education","gender","recruitment_channel","region","age","KPIs_met >80%","awards_won?"]]
data_xs_9 = Data.loc[:,["region","age","KPIs_met >80%","awards_won?"]]
data_xs_10 = Data.loc[:,["department","education","gender","recruitment_channel","previous_year_rating","avg_training_score","length_of_service","no_of_trainings"]]

In [0]:
categorical_features_indices_5 = np.array([0,1,2,3,4,5,6,7])
categorical_features_indices_6 = np.array([0,1])
categorical_features_indices_7 = np.array([0,1,2,3,4,5,6])
categorical_features_indices_8 = np.array([0,1,2,3,4,5,6])
categorical_features_indices_9 = np.array([0,1,2,3])
categorical_features_indices_10 = np.array([0,1,2,3])

In [0]:
data_xs.head()


In [0]:
def model_evalution(model):
  print("####################### model Evalution started #######################")
  train_pre = model.predict(x_train)
  test_pre = model.predict(x_test)
  train_pro = model.predict_proba(x_train)
  test_pro = model.predict_proba(x_test)
  
  print("Train Accuracy: {0} \t Test Accuracy: {1}".format(accuracy_score(y_train, train_pre),accuracy_score(y_test,test_pre)))
  print("Train Loss: {0} \t Test Loss: {1}".format(mean_squared_error(y_train, train_pre),mean_squared_error(y_test,test_pre)))
  print("Train AUC: {0} \t Test AUC: {1}".format(roc_auc_score(y_train, train_pro[:,1]),roc_auc_score(y_test,test_pro[:,1])))
  print("Train f1: {0} \t Test F1: {1}".format(f1_score(y_train, train_pre),f1_score(y_test,test_pre)))
  print("Train Confusion Matrix: \n{0} \n Test Confusion Matrix: \n{1}".format(confusion_matrix(y_train, train_pre),confusion_matrix(y_test,test_pre)))
  #f1_score
  

In [0]:
x_train,x_test,y_train,y_test = train_test_split(data_xs,Data["is_promoted"], test_size = 0.3)

In [0]:
x_train_1,x_test_1,y_train_1,y_test_1 = train_test_split(data_xs_1,Data["is_promoted"], test_size = 0.3,random_state=SEED)
x_train_2,x_test_2,y_train_2,y_test_2 = train_test_split(data_xs_2,Data["is_promoted"], test_size = 0.3,random_state=SEED)
x_train_3,x_test_3,y_train_3,y_test_3 = train_test_split(data_xs_3,Data["is_promoted"], test_size = 0.3,random_state=SEED)
x_train_4,x_test_4,y_train_4,y_test_4 = train_test_split(data_xs_4,Data["is_promoted"], test_size = 0.3,random_state=SEED)
x_train_5,x_test_5,y_train_5,y_test_5 = train_test_split(data_xs_5,Data["is_promoted"], test_size = 0.3,random_state=SEED)
x_train_6,x_test_6,y_train_6,y_test_6 = train_test_split(data_xs_6,Data["is_promoted"], test_size = 0.3,random_state=SEED)
x_train_7,x_test_7,y_train_7,y_test_7 = train_test_split(data_xs_7,Data["is_promoted"], test_size = 0.3,random_state=SEED)
x_train_8,x_test_8,y_train_8,y_test_8 = train_test_split(data_xs_8,Data["is_promoted"], test_size = 0.3,random_state=SEED)
x_train_9,x_test_9,y_train_9,y_test_9 = train_test_split(data_xs_9,Data["is_promoted"], test_size = 0.3,random_state=SEED)
x_train_10,x_test_10,y_train_10,y_test_10 = train_test_split(data_xs_10,Data["is_promoted"], test_size = 0.3,random_state=SEED)

In [0]:
params = {'iterations':[300],
          'eval_metric':["F1"],
          'learning_rate':[1,0.1,0.01],
          'depth':[2],
          'verbose':[True],
          'class_weights':[[1,9]],
          'l2_leaf_reg':[10,1,0.1,0.01,0.001,0.0001],
          'bagging_temperature':[0,0.1,0.5,0,0.01,0.07],
         }

In [0]:
params = {'iterations':[50],
          'eval_metric':["F1"],
          #'learning_rate':[1,0.1,0.01],
          'depth':[2],
          'verbose':[True],
          'class_weights':[[1,9]],
          #'l2_leaf_reg':[10,1,0.1,0.01,0.001,0.0001],
          #'bagging_temperature':[0,0.1,0.5,0,0.01,0.07],
         }

In [0]:
rf_model = CatBoostClassifier()
grid = GridSearchCV(rf_model,params,scoring="accuracy")
grid.fit(x_train, y_train,
         cat_features=categorical_features_indices,
         eval_set=(x_test, y_test),)
print(grid.best_params_)
model_evalution(grid)

In [239]:
del(catboost_model_1,catboost_model_2,catboost_model_3,catboost_model_4,catboost_model_5,catboost_model_6,catboost_model_7,catboost_model_8,catboost_model_9,catboost_model_10)

NameError: ignored

In [0]:
catboost_model_1 = CatBoostClassifier(class_weights=[1,4],iterations=100,bagging_temperature=0.2,depth=3).fit(x_train_1, y_train_1)
catboost_model_2 = CatBoostClassifier(class_weights=[1,4],iterations=100,bagging_temperature=0.2,depth=3).fit(x_train_2, y_train_2)
catboost_model_3 = CatBoostClassifier(class_weights=[1,4],iterations=100,bagging_temperature=0.2,depth=3).fit(x_train_3, y_train_3)
catboost_model_4 = CatBoostClassifier(class_weights=[1,4],iterations=100,bagging_temperature=0.2,depth=3).fit(x_train_4, y_train_4)
catboost_model_5 = CatBoostClassifier(class_weights=[1,4],iterations=100,bagging_temperature=0.2,depth=3).fit(x_train_5, y_train_5,cat_features=categorical_features_indices_5)
catboost_model_6 = CatBoostClassifier(class_weights=[1,4],iterations=100,bagging_temperature=0.2,depth=3).fit(x_train_6, y_train_6,cat_features=categorical_features_indices_6)
catboost_model_7 = CatBoostClassifier(class_weights=[1,4],iterations=100,bagging_temperature=0.2,depth=3).fit(x_train_7, y_train_7,cat_features=categorical_features_indices_7)
catboost_model_8 = CatBoostClassifier(class_weights=[1,4],iterations=100,bagging_temperature=0.2,depth=3).fit(x_train_8, y_train_8,cat_features=categorical_features_indices_8)
catboost_model_9 = CatBoostClassifier(class_weights=[1,4],iterations=100,bagging_temperature=0.2,depth=3).fit(x_train_9, y_train_9,cat_features=categorical_features_indices_9)
catboost_model_10 = CatBoostClassifier(class_weights=[1,4],iterations=100,bagging_temperature=0.2,depth=3).fit(x_train_10, y_train_10,cat_features=categorical_features_indices_10)


In [0]:
catboost_model_1 = CatBoostClassifier(class_weights=[1,9],iterations=50,bagging_temperature=0.3,depth=3,l2_leaf_reg=10
                                     ).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_2 = CatBoostClassifier(class_weights=[1,5],iterations=100,bagging_temperature=0.2,depth=3,l2_leaf_reg=10
                                     ).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_3 = CatBoostClassifier(class_weights=[1,3],iterations=500,bagging_temperature=0.2,depth=3,l2_leaf_reg=10
                                     ).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_4 = CatBoostClassifier(class_weights=[1,3],iterations=100,bagging_temperature=0.2,depth=3,l2_leaf_reg=10
                                     ).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_5 = CatBoostClassifier(class_weights=[1,2],iterations=500,bagging_temperature=0.2,depth=3,l2_leaf_reg=10
                                     ).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_6 = CatBoostClassifier(class_weights=[1,2],iterations=100,bagging_temperature=0.2,depth=3,l2_leaf_reg=10
                                     ).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_7 = CatBoostClassifier(class_weights=[1,1],iterations=300,bagging_temperature=0.2,depth=3,l2_leaf_reg=10
                                     ).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_8 = CatBoostClassifier(class_weights=[1,1],iterations=100,bagging_temperature=0.2,depth=1,l2_leaf_reg=10
                                     ).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_9 = CatBoostClassifier(class_weights=[1,3],iterations=100,bagging_temperature=0.2,depth=2,l2_leaf_reg=10
                                     ).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_10 = CatBoostClassifier(class_weights=[1,4],iterations=300,bagging_temperature=0.2,depth=3,l2_leaf_reg=10
                                      ).fit(x_train, y_train,cat_features=categorical_features_indices)


In [0]:
catboost_model_1 = CatBoostClassifier(class_weights=[1,9],iterations=500,bagging_temperature=0.5,depth=4,l2_leaf_reg=50
                                     ).fit(x_train, y_train,cat_features=categorical_features_indices,eval_set=(x_test, y_test))
catboost_model_2 = CatBoostClassifier(class_weights=[1,5],iterations=500,bagging_temperature=0.2,depth=3,l2_leaf_reg=50
                                     ).fit(x_train, y_train,cat_features=categorical_features_indices,eval_set=(x_test, y_test))
catboost_model_3 = CatBoostClassifier(class_weights=[1,3],iterations=500,bagging_temperature=0.2,depth=2,l2_leaf_reg=50
                                     ).fit(x_train, y_train,cat_features=categorical_features_indices,eval_set=(x_test, y_test))
catboost_model_4 = CatBoostClassifier(class_weights=[1,3],iterations=500,bagging_temperature=0.2,depth=3,l2_leaf_reg=50
                                     ).fit(x_train, y_train,cat_features=categorical_features_indices,eval_set=(x_test, y_test))
catboost_model_5 = CatBoostClassifier(class_weights=[1,2],iterations=500,bagging_temperature=0.2,depth=3,l2_leaf_reg=50
                                     ).fit(x_train, y_train,cat_features=categorical_features_indices,eval_set=(x_test, y_test))
catboost_model_6 = CatBoostClassifier(class_weights=[1,2],iterations=500,bagging_temperature=0.2,depth=2,l2_leaf_reg=50
                                     ).fit(x_train, y_train,cat_features=categorical_features_indices,eval_set=(x_test, y_test))
catboost_model_7 = CatBoostClassifier(class_weights=[1,1],iterations=500,bagging_temperature=0.2,depth=2,l2_leaf_reg=50
                                     ).fit(x_train, y_train,cat_features=categorical_features_indices,eval_set=(x_test, y_test))
catboost_model_8 = CatBoostClassifier(class_weights=[1,1],iterations=500,bagging_temperature=0.2,depth=3,l2_leaf_reg=50
                                     ).fit(x_train, y_train,cat_features=categorical_features_indices,eval_set=(x_test, y_test))
catboost_model_9 = CatBoostClassifier(class_weights=[1,3],iterations=500,bagging_temperature=0.2,depth=2,l2_leaf_reg=50
                                     ).fit(x_train, y_train,cat_features=categorical_features_indices,eval_set=(x_test, y_test))
catboost_model_10 = CatBoostClassifier(class_weights=[1,3],iterations=500,bagging_temperature=0.2,depth=3,l2_leaf_reg=50
                                      ).fit(x_train, y_train,cat_features=categorical_features_indices,eval_set=(x_test, y_test))


In [0]:
catboost_model_1 = CatBoostClassifier(class_weights=[1,9],iterations=10,bagging_temperature=0.2,depth=3,l2_leaf_reg=50,learning_rate=1).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_2 = CatBoostClassifier(class_weights=[1,8],iterations=50,bagging_temperature=0.2,depth=3,l2_leaf_reg=50,learning_rate=1).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_3 = CatBoostClassifier(class_weights=[1,7],iterations=100,bagging_temperature=0.2,depth=3,l2_leaf_reg=50,learning_rate=0.9).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_4 = CatBoostClassifier(class_weights=[1,6],iterations=150,bagging_temperature=0.2,depth=3,l2_leaf_reg=50,learning_rate=0.7).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_5 = CatBoostClassifier(class_weights=[1,5],iterations=200,bagging_temperature=0.2,depth=3,l2_leaf_reg=50,learning_rate=0.6).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_6 = CatBoostClassifier(class_weights=[1,4],iterations=250,bagging_temperature=0.2,depth=3,l2_leaf_reg=50,learning_rate=0.5).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_7 = CatBoostClassifier(class_weights=[1,3],iterations=300,bagging_temperature=0.2,depth=3,l2_leaf_reg=50,learning_rate=0.3).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_8 = CatBoostClassifier(class_weights=[1,2],iterations=400,bagging_temperature=0.2,depth=3,l2_leaf_reg=50,learning_rate=0.1).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_9 = CatBoostClassifier(class_weights=[1,1],iterations=500,bagging_temperature=0.2,depth=3,l2_leaf_reg=50,learning_rate=0.01).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_10 = CatBoostClassifier(class_weights=[1,2],iterations=600,bagging_temperature=0.2,depth=3,l2_leaf_reg=50,learning_rate=0.001).fit(x_train, y_train,cat_features=categorical_features_indices)


In [0]:
catboost_model_1 = CatBoostClassifier(class_weights=[1,3],iterations=10,bagging_temperature=0.2,depth=3,l2_leaf_reg=50,learning_rate=1).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_2 = CatBoostClassifier(class_weights=[1,3],iterations=50,bagging_temperature=0.2,depth=3,l2_leaf_reg=50,learning_rate=1).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_3 = CatBoostClassifier(class_weights=[1,3],iterations=100,bagging_temperature=0.2,depth=3,l2_leaf_reg=50,learning_rate=0.9).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_4 = CatBoostClassifier(class_weights=[1,3],iterations=150,bagging_temperature=0.2,depth=3,l2_leaf_reg=50,learning_rate=0.7).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_5 = CatBoostClassifier(class_weights=[1,3],iterations=200,bagging_temperature=0.2,depth=3,l2_leaf_reg=50,learning_rate=0.6).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_6 = CatBoostClassifier(class_weights=[1,3],iterations=250,bagging_temperature=0.2,depth=3,l2_leaf_reg=50,learning_rate=0.5).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_7 = CatBoostClassifier(class_weights=[1,3],iterations=300,bagging_temperature=0.2,depth=3,l2_leaf_reg=50,learning_rate=0.3).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_8 = CatBoostClassifier(class_weights=[1,3],iterations=400,bagging_temperature=0.2,depth=3,l2_leaf_reg=50,learning_rate=0.1).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_9 = CatBoostClassifier(class_weights=[1,3],iterations=500,bagging_temperature=0.2,depth=3,l2_leaf_reg=50,learning_rate=0.01).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_10 = CatBoostClassifier(class_weights=[1,3],iterations=600,bagging_temperature=0.2,depth=3,l2_leaf_reg=50,learning_rate=0.001).fit(x_train, y_train,cat_features=categorical_features_indices)


In [0]:
catboost_model_1 = CatBoostClassifier(class_weights=[1,3],iterations=10,bagging_temperature=0.2).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_2 = CatBoostClassifier(class_weights=[1,3],iterations=20,bagging_temperature=0.2).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_3 = CatBoostClassifier(class_weights=[1,3],iterations=30,bagging_temperature=0.2).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_4 = CatBoostClassifier(class_weights=[1,3],iterations=50,bagging_temperature=0.2).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_5 = CatBoostClassifier(class_weights=[1,3],iterations=60,bagging_temperature=0.2).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_6 = CatBoostClassifier(class_weights=[1,3],iterations=70,bagging_temperature=0.2).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_7 = CatBoostClassifier(class_weights=[1,3],iterations=80,bagging_temperature=0.2).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_8 = CatBoostClassifier(class_weights=[1,3],iterations=90,bagging_temperature=0.2).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_9 = CatBoostClassifier(class_weights=[1,3],iterations=100,bagging_temperature=0.2).fit(x_train, y_train,cat_features=categorical_features_indices)
catboost_model_10 = CatBoostClassifier(class_weights=[1,3],iterations=110,bagging_temperature=0.2).fit(x_train, y_train,cat_features=categorical_features_indices)


In [241]:
catboost_model_1 = CatBoostClassifier(iterations=100,verbose=True,eval_metric="F1",learning_rate=0.5,class_weights=[1,3],depth=3,l2_leaf_reg=50,bagging_temperature=0.5
                                     ).fit(x_train, y_train,cat_features=categorical_features_indices,eval_set=(x_test, y_test))
catboost_model_2 = CatBoostClassifier(iterations=200,verbose=True,eval_metric="F1",learning_rate=0.5,class_weights=[1,3],depth=3,l2_leaf_reg=50,bagging_temperature=0.5
                                     ).fit(x_train, y_train,cat_features=categorical_features_indices,eval_set=(x_test, y_test))
catboost_model_3 = CatBoostClassifier(iterations=300,verbose=True,eval_metric="F1",learning_rate=0.5,class_weights=[1,3],depth=3,l2_leaf_reg=50,bagging_temperature=0.5
                                     ).fit(x_train, y_train,cat_features=categorical_features_indices,eval_set=(x_test, y_test))
catboost_model_4 = CatBoostClassifier(iterations=500,verbose=True,eval_metric="F1",learning_rate=0.5,class_weights=[1,3],depth=3,l2_leaf_reg=50,bagging_temperature=0.5
                                     ).fit(x_train, y_train,cat_features=categorical_features_indices,eval_set=(x_test, y_test))

0:	learn: 0.2033010	test: 0.2076503	best: 0.2076503 (0)	total: 134ms	remaining: 13.3s
1:	learn: 0.5185934	test: 0.5330411	best: 0.5330411 (1)	total: 258ms	remaining: 12.6s
2:	learn: 0.5153807	test: 0.5317623	best: 0.5330411 (1)	total: 382ms	remaining: 12.3s
3:	learn: 0.5150290	test: 0.5329015	best: 0.5330411 (1)	total: 479ms	remaining: 11.5s
4:	learn: 0.5133289	test: 0.5262076	best: 0.5330411 (1)	total: 597ms	remaining: 11.4s
5:	learn: 0.5154533	test: 0.5271795	best: 0.5330411 (1)	total: 717ms	remaining: 11.2s
6:	learn: 0.5167780	test: 0.5292308	best: 0.5330411 (1)	total: 848ms	remaining: 11.3s
7:	learn: 0.5187244	test: 0.5347848	best: 0.5347848 (7)	total: 962ms	remaining: 11.1s
8:	learn: 0.5227818	test: 0.5344623	best: 0.5347848 (7)	total: 1.09s	remaining: 11.1s
9:	learn: 0.5276295	test: 0.5369218	best: 0.5369218 (9)	total: 1.21s	remaining: 10.9s
10:	learn: 0.5259952	test: 0.5364539	best: 0.5369218 (9)	total: 1.33s	remaining: 10.8s
11:	learn: 0.5264836	test: 0.5384874	best: 0.5384874 

In [0]:
X_1 = catboost_model_1.predict(x_train)
X_2 = catboost_model_2.predict(x_train)
X_3 = catboost_model_3.predict(x_train)
X_4 = catboost_model_4.predict(x_train)
X_5 = catboost_model_5.predict(x_train)
X_6 = catboost_model_6.predict(x_train)
X_7 = catboost_model_7.predict(x_train)
X_8 = catboost_model_8.predict(x_train)
X_9 = catboost_model_9.predict(x_train)
X_10 = catboost_model_10.predict(x_train)

In [0]:
X_1 = catboost_model_1.predict(x_train)
X_2 = catboost_model_2.predict(x_train)
X_3 = catboost_model_3.predict(x_train)
X_4 = catboost_model_4.predict(x_train)


In [0]:
X_1 = catboost_model_1.predict(x_train_1)
X_2 = catboost_model_2.predict(x_train_2)
X_3 = catboost_model_3.predict(x_train_3)
X_4 = catboost_model_4.predict(x_train_4)
X_5 = catboost_model_5.predict(x_train_5)
X_6 = catboost_model_6.predict(x_train_6)
X_7 = catboost_model_7.predict(x_train_7)
X_8 = catboost_model_8.predict(x_train_8)
X_9 = catboost_model_9.predict(x_train_9)
X_10 = catboost_model_10.predict(x_train_10)

In [0]:
X_t_1 = catboost_model_1.predict(x_test)
X_t_2 = catboost_model_2.predict(x_test)
X_t_3 = catboost_model_3.predict(x_test)
X_t_4 = catboost_model_4.predict(x_test)
X_t_5 = catboost_model_5.predict(x_test)
X_t_6 = catboost_model_6.predict(x_test)
X_t_7 = catboost_model_7.predict(x_test)
X_t_8 = catboost_model_8.predict(x_test)
X_t_9 = catboost_model_9.predict(x_test)
X_t_10 = catboost_model_10.predict(x_test)

X_t = np.vstack([X_t_1,X_t_2,X_t_3,X_t_4,X_t_5,X_t_6,X_t_7,X_t_8,X_t_9,X_t_10]).T

In [0]:
X_t_1 = catboost_model_1.predict(x_test_1)
X_t_2 = catboost_model_2.predict(x_test_2)
X_t_3 = catboost_model_3.predict(x_test_3)
X_t_4 = catboost_model_4.predict(x_test_4)
X_t_5 = catboost_model_5.predict(x_test_5)
X_t_6 = catboost_model_6.predict(x_test_6)
X_t_7 = catboost_model_7.predict(x_test_7)
X_t_8 = catboost_model_8.predict(x_test_8)
X_t_9 = catboost_model_9.predict(x_test_9)
X_t_10 = catboost_model_10.predict(x_test_10)

X_t = np.vstack([X_t_1,X_t_2,X_t_3,X_t_4,X_t_5,X_t_6,X_t_7,X_t_8,X_t_9,X_t_10]).T

In [0]:
X_t_1 = catboost_model_1.predict(x_test)
X_t_2 = catboost_model_2.predict(x_test)
X_t_3 = catboost_model_3.predict(x_test)
X_t_4 = catboost_model_4.predict(x_test)


X_t = np.vstack([X_t_1,X_t_2,X_t_3,X_t_4]).T

In [0]:
X = np.vstack([X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10]).T

In [0]:
X = np.vstack([X_1,X_2,X_3,X_4]).T

In [144]:
print(X.shape)
print(X_t.shape)

(38365, 10)
(16443, 10)


In [0]:
def model_evalution_stack(model):
  print("####################### model Evalution started #######################")
  train_pre = model.predict(X)
  test_pre = model.predict(X_t)
  train_pro = model.predict_proba(X)
  test_pro = model.predict_proba(X_t)
  
  print("Train Accuracy: {0} \t Test Accuracy: {1}".format(accuracy_score(y_train, train_pre),accuracy_score(y_test,test_pre)))
  print("Train Loss: {0} \t Test Loss: {1}".format(mean_squared_error(y_train, train_pre),mean_squared_error(y_test,test_pre)))
  print("Train AUC: {0} \t Test AUC: {1}".format(roc_auc_score(y_train, train_pro[:,1]),roc_auc_score(y_test,test_pro[:,1])))
  print("Train f1: {0} \t Test F1: {1}".format(f1_score(y_train, train_pre),f1_score(y_test,test_pre)))
  print("Train Confusion Matrix: \n{0} \n Test Confusion Matrix: \n{1}".format(confusion_matrix(y_train, train_pre),confusion_matrix(y_test,test_pre)))
  #f1_score
  

In [0]:
def maxVoting(X):
  
  #return list(map(lambda i:0 if np.count_nonzero(i == 0) > np.count_nonzero(i == 1) else 1,X))
  return list(map(lambda i:0 if np.count_nonzero(i == 0) > 2 else 1,X))

Y_pre_max = maxVoting(X)
Y_t_pre_max = maxVoting(X_t)

In [246]:
  print("####################### model Evalution started #######################")
  train_pre = Y_pre_max
  test_pre = Y_t_pre_max
  
  print("Train Accuracy: {0} \t Test Accuracy: {1}".format(accuracy_score(y_train, train_pre),accuracy_score(y_test,test_pre)))
  print("Train Loss: {0} \t Test Loss: {1}".format(mean_squared_error(y_train, train_pre),mean_squared_error(y_test,test_pre)))
  #print("Train AUC: {0} \t Test AUC: {1}".format(roc_auc_score(y_train, train_pro[:,1]),roc_auc_score(y_test,test_pro[:,1])))
  print("Train f1: {0} \t Test F1: {1}".format(f1_score(y_train, train_pre),f1_score(y_test,test_pre)))
  print("Train Confusion Matrix: \n{0} \n Test Confusion Matrix: \n{1}".format(confusion_matrix(y_train, train_pre),confusion_matrix(y_test,test_pre)))

####################### model Evalution started #######################
Train Accuracy: 0.9342369347061124 	 Test Accuracy: 0.9292099981755154
Train Loss: 0.06576306529388766 	 Test Loss: 0.07079000182448458
Train f1: 0.5506678539626002 	 Test F1: 0.5221674876847291
Train Confusion Matrix: 
[[34296   811]
 [ 1712  1546]] 
 Test Confusion Matrix: 
[[14643   390]
 [  774   636]]


In [0]:
 del(catboost_model)
catboost_model = CatBoostClassifier(iterations=10,verbose=True,eval_metric="F1",learning_rate=0.5,class_weights=[1,2],depth=2,l2_leaf_reg=50,bagging_temperature=0.4)
catboost_model.fit(X, y_train,eval_set=(X_t, y_test))
model_evalution_stack(catboost_model)

In [0]:
def model_evalution(model,x_train_temp=x_train,x_test_temp=x_train):
  print("####################### model Evalution started #######################")
  train_pre = model.predict(x_train_temp)
  test_pre = model.predict(x_test_temp)
  train_pro = model.predict_proba(x_train_temp)
  test_pro = model.predict_proba(x_test_temp)
  
  print("Train Accuracy: {0} \t Test Accuracy: {1}".format(accuracy_score(y_train, train_pre),accuracy_score(y_test,test_pre)))
  print("Train Loss: {0} \t Test Loss: {1}".format(mean_squared_error(y_train, train_pre),mean_squared_error(y_test,test_pre)))
  print("Train AUC: {0} \t Test AUC: {1}".format(roc_auc_score(y_train, train_pro[:,1]),roc_auc_score(y_test,test_pro[:,1])))
  print("Train f1: {0} \t Test F1: {1}".format(f1_score(y_train, train_pre),f1_score(y_test,test_pre)))
  print("Train Confusion Matrix: \n{0} \n Test Confusion Matrix: \n{1}".format(confusion_matrix(y_train, train_pre),confusion_matrix(y_test,test_pre)))
  #f1_score
  

In [247]:
model_evalution(catboost_model_1)
model_evalution(catboost_model_2)
model_evalution(catboost_model_3)
model_evalution(catboost_model_4)
model_evalution(catboost_model_5)
model_evalution(catboost_model_6)
model_evalution(catboost_model_7)
model_evalution(catboost_model_8)
model_evalution(catboost_model_9,)
model_evalution(catboost_model_10)

####################### model Evalution started #######################
Train Accuracy: 0.9335331682523134 	 Test Accuracy: 0.9289059174116645
Train Loss: 0.0664668317476867 	 Test Loss: 0.07109408258833547
Train AUC: 0.9172992806014789 	 Test AUC: 0.9078641881477769
Train f1: 0.5436649964209019 	 Test F1: 0.5191279308926369
Train Confusion Matrix: 
[[34296   811]
 [ 1739  1519]] 
 Test Confusion Matrix: 
[[14643   390]
 [  779   631]]
####################### model Evalution started #######################
Train Accuracy: 0.936165776098006 	 Test Accuracy: 0.9322508058140242
Train Loss: 0.063834223901994 	 Test Loss: 0.06774919418597579
Train AUC: 0.9170936870834044 	 Test AUC: 0.9075235663573236
Train f1: 0.5431822421190076 	 Test F1: 0.5210662080825451
Train Confusion Matrix: 
[[34460   647]
 [ 1802  1456]] 
 Test Confusion Matrix: 
[[14723   310]
 [  804   606]]
####################### model Evalution started #######################
Train Accuracy: 0.9342890655545419 	 Test Accuracy

NameError: ignored

In [89]:
model_evalution(catboost_model_1,x_train_1,x_test_1)
model_evalution(catboost_model_2,x_train_2,x_test_2)
model_evalution(catboost_model_3,x_train_3,x_test_3)
model_evalution(catboost_model_4,x_train_4,x_test_4)
model_evalution(catboost_model_5,x_train_5,x_test_5)
model_evalution(catboost_model_6,x_train_6,x_test_6)
model_evalution(catboost_model_7,x_train_7,x_test_7)
model_evalution(catboost_model_8,x_train_8,x_test_8)
model_evalution(catboost_model_9,x_train_9,x_test_9)
model_evalution(catboost_model_10,x_train_10,x_test_10)

####################### model Evalution started #######################
Train Accuracy: 0.9181545679655937 	 Test Accuracy: 0.9183847229824241
Train Loss: 0.08184543203440636 	 Test Loss: 0.08161527701757587
Train AUC: 0.7385392094377461 	 Test AUC: 0.7323725159449408
Train f1: 0.24519230769230768 	 Test F1: 0.24009060022650056
Train Confusion Matrix: 
[[34715   368]
 [ 2772   510]] 
 Test Confusion Matrix: 
[[14889   168]
 [ 1174   212]]
####################### model Evalution started #######################
Train Accuracy: 0.9125765671836309 	 Test Accuracy: 0.9147965699689837
Train Loss: 0.08742343281636909 	 Test Loss: 0.08520343003101624
Train AUC: 0.7359923458608292 	 Test AUC: 0.7323153737778164
Train f1: 0.27808867843306073 	 Test F1: 0.28190671450538185
Train Confusion Matrix: 
[[34365   718]
 [ 2636   646]] 
 Test Confusion Matrix: 
[[14767   290]
 [ 1111   275]]
####################### model Evalution started #######################
Train Accuracy: 0.920109474781702 	 Test A

In [0]:
del(catboost_model)

In [179]:
#catboost_model = CatBoostClassifier(class_weights=[1,9])
catboost_model = CatBoostClassifier(iterations=200,verbose=True,eval_metric="F1",learning_rate=1,class_weights=[1,3],depth=3,l2_leaf_reg=40,bagging_temperature=0.2)
catboost_model.fit(x_train, y_train,cat_features=categorical_features_indices,eval_set=(x_test, y_test))
model_evalution(catboost_model)

0:	learn: 0.1996516	test: 0.2162052	best: 0.2162052 (0)	total: 90.3ms	remaining: 18s
1:	learn: 0.5266169	test: 0.5379242	best: 0.5379242 (1)	total: 179ms	remaining: 17.7s
2:	learn: 0.5323521	test: 0.5363576	best: 0.5379242 (1)	total: 268ms	remaining: 17.6s
3:	learn: 0.5382432	test: 0.5503134	best: 0.5503134 (3)	total: 357ms	remaining: 17.5s
4:	learn: 0.5417242	test: 0.5494215	best: 0.5503134 (3)	total: 440ms	remaining: 17.2s
5:	learn: 0.5356317	test: 0.5449041	best: 0.5503134 (3)	total: 525ms	remaining: 17s
6:	learn: 0.5309245	test: 0.5487620	best: 0.5503134 (3)	total: 600ms	remaining: 16.6s
7:	learn: 0.5316616	test: 0.5490196	best: 0.5503134 (3)	total: 676ms	remaining: 16.2s
8:	learn: 0.5358798	test: 0.5500083	best: 0.5503134 (3)	total: 760ms	remaining: 16.1s
9:	learn: 0.5370577	test: 0.5573554	best: 0.5573554 (9)	total: 838ms	remaining: 15.9s
10:	learn: 0.5388623	test: 0.5555372	best: 0.5573554 (9)	total: 918ms	remaining: 15.8s
11:	learn: 0.5389746	test: 0.5555372	best: 0.5573554 (9)

In [63]:
catboost_model.feature_importances_

array([12.30221478,  3.1660155 ,  1.03005414,  0.10623195,  1.05865546,
        0.17229958,  4.28291136,  6.02134667,  2.61133227, 33.25769803,
        2.07777626, 33.913464  ])

In [0]:
cat_features=categorical_features_indices,eval_set=(x_test, y_test)

In [0]:
Data_test = pd.read_csv("test_2umaH9m.csv")

In [128]:
print(Data_test.shape)
print(Data_test.columns)

(23490, 13)
Index(['employee_id', 'department', 'region', 'education', 'gender',
       'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'KPIs_met >80%', 'awards_won?',
       'avg_training_score'],
      dtype='object')


In [129]:
print(Data_test.describe())

        employee_id  no_of_trainings           age  previous_year_rating  \
count  23490.000000     23490.000000  23490.000000          21678.000000   
mean   39041.399149         1.254236     34.782929              3.339146   
std    22640.809201         0.600910      7.679492              1.263294   
min        3.000000         1.000000     20.000000              1.000000   
25%    19370.250000         1.000000     29.000000              3.000000   
50%    38963.500000         1.000000     33.000000              3.000000   
75%    58690.000000         1.000000     39.000000              4.000000   
max    78295.000000         9.000000     60.000000              5.000000   

       length_of_service  KPIs_met >80%   awards_won?  avg_training_score  
count       23490.000000   23490.000000  23490.000000        23490.000000  
mean            5.810387       0.358834      0.022776           63.263133  
std             4.207917       0.479668      0.149191           13.411750  
min        

In [0]:
values={"no_of_trainings":0,"previous_year_rating":0,"gender":"O","education":"O"}
Data_test.fillna(value=values,inplace=True)

In [131]:
Data_test.isna().values.any()


False

In [132]:
Data_test.count()

employee_id             23490
department              23490
region                  23490
education               23490
gender                  23490
recruitment_channel     23490
no_of_trainings         23490
age                     23490
previous_year_rating    23490
length_of_service       23490
KPIs_met >80%           23490
awards_won?             23490
avg_training_score      23490
dtype: int64

In [133]:
Data_test.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,0.0,1,1,0,77
1,74430,HR,region_4,Bachelor's,f,other,1,31,3.0,5,0,0,51
2,72255,Sales & Marketing,region_13,Bachelor's,m,other,1,31,1.0,4,0,0,47
3,38562,Procurement,region_2,Bachelor's,f,other,3,31,2.0,9,0,0,65
4,64486,Finance,region_29,Bachelor's,m,sourcing,1,30,4.0,7,0,0,61


In [0]:
data_test_xs = Data_test.drop(['employee_id'],axis=1)
#data_test_xs = Data_test.drop(['employee_id',"education","gender","recruitment_channel","region"],axis=1)

In [0]:
y_test_pre = catboost_model.predict(data_test_xs)

In [0]:
X_t_1_test = catboost_model_1.predict(data_test_xs)
X_t_2_test = catboost_model_2.predict(data_test_xs)
X_t_3_test = catboost_model_3.predict(data_test_xs)
X_t_4_test = catboost_model_4.predict(data_test_xs)
X_t_5_test = catboost_model_5.predict(data_test_xs)
X_t_6_test = catboost_model_6.predict(data_test_xs)
X_t_7_test = catboost_model_7.predict(data_test_xs)
X_t_8_test = catboost_model_8.predict(data_test_xs)
X_t_9_test = catboost_model_9.predict(data_test_xs)
X_t_10_test = catboost_model_10.predict(data_test_xs)

X_t_test = np.vstack([X_t_1_test,X_t_2_test,X_t_3_test,X_t_4_test,X_t_5_test,X_t_6_test,X_t_7_test,X_t_8_test,X_t_9_test,X_t_10_test]).T

In [0]:
y_test_pre = catboost_model.predict(X_t_test)

In [0]:
Data_test["is_promoted"] = y_test_pre

In [0]:
Data_test.to_csv("WNS_catBoost_Test_03_stacked.csv",columns=["employee_id","is_promoted"],index=False)

In [0]:
from google.colab import files
files.download("WNS_catBoost_Test_03_stacked.csv")